Setup Paths

In [1]:
WORKSPACE_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace'
SCRIPTS_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\scripts'
APIMODEL_PATH = r'C:\Users\Admin\Downloads\tensorflow\models'
ANNOTATION_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\annotations'
IMAGE_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\images'
MODEL_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models'
PRETRAINED_MODEL_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\pre-trained-models'
CONFIG_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config'
CHECKPOINT_PATH = r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models\my_ssd_mobnet\checkpoint'

Create Label Map

In [2]:
labels = [{'name':'cylinder', 'id':1}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

Create TF records

In [3]:
!python {SCRIPTS_PATH + '\generate_tfrecord.py'} -x {IMAGE_PATH + '\\train'} -l {ANNOTATION_PATH + '\label_map.pbtxt'} -o {ANNOTATION_PATH + '\\train.record'}
!python {SCRIPTS_PATH + '\generate_tfrecord.py'} -x{IMAGE_PATH + '\\test'} -l {ANNOTATION_PATH + '\label_map.pbtxt'} -o {ANNOTATION_PATH + '\\test.record'}

Successfully created the TFRecord file: C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\annotations\train.record
Successfully created the TFRecord file: C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\annotations\test.record


Copy Model Config to Training Folder

In [4]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [5]:
!mkdir {r'C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models\\'+ CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config'} {MODEL_PATH+'\\'+ CUSTOM_MODEL_NAME}

Update Config For Transfer Learning

In [6]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
CONFIG_PATH = MODEL_PATH+'\\my_ssd_mobnet\\pipeline.config'

In [8]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [11]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\checkpoint\ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '\label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '\\train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '\label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '\\test.record']

In [12]:
pipeline_config

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      arg

In [13]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

Train the model

In [14]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH,MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python C:\Users\Admin\Downloads\tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models/my_ssd_mobnet --pipeline_config_path=C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


Load Train Model From Checkpoint

In [1]:
!python C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\scripts\exporter_main_v2.py --input_type image_tensor --pipeline_config_path C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --trained_checkpoint_dir C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\models\my_ssd_mobnet --output_directory C:\Users\Admin\Downloads\RealTimeObjectDetection-main\Tensorflow\workspace\exported_models\lpg_model

2021-07-22 11:50:06.476541: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-22 11:50:06.477688: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-07-22 11:50:18.801095: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2021-07-22 11:50:18.801132: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-22 11:50:18.808123: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-SFFOMF0
2021-07-22 11:50:18.808266: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-SFFOMF0
2021-07-22 11:50:18.812491: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is